# Imports

In [1]:
import os
import pywt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import numpy as np
import random
from pytorch_lightning.loggers import WandbLogger
import wandb
import auraloss
import collections
from tqdm import tqdm
import pretty_midi
import matplotlib.pyplot as plt
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
print(torch.cuda.is_available())
import plotly.graph_objects as go
from torch.optim import lr_scheduler
from IPython.display import Audio
from torchaudio.transforms import Fade
import musdb
import museval
import gc

True


# Set Seeds

In [2]:
seed_value = 3407
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.set_float32_matmul_precision('high')

# Construct Teh Datas

In [3]:
path = "D:/Github/phd-drum-sep/Data/musdb18hq/"

In [4]:
os.listdir(path)

['test', 'train']

In [5]:
train = list(os.listdir(path+'train'))
test = list(os.listdir(path+'test'))

In [6]:
sources = ['drum', 'bass', 'other', 'vocals']

In [7]:
all_scenes = {}
counter = 0
sample_rate = 44100
segment_length = sample_rate * 1

for idx, val in tqdm(enumerate(test)):
    p = path + 'test/' + val + "/"
    info = torchaudio.info(f"{p}mixture.wav")
    seconds = info.num_frames // 44100
    for i in range(0, seconds - 1, 1):
        start_point = i * 44100
        if start_point + 44100 < info.num_frames:
            all_scenes[counter] = {'music_path': p, 'start_point': start_point, 'length': 44100, 'frames' : info.num_frames}
            counter += 1

50it [00:00, 4166.72it/s]


In [8]:
def turn_transcription_into_roll(transcription, frames):
    # Determine your sampling frequency (frames per second)
    fs = 44100
    
    piano_roll_length = int(frames)
    
    # Initialize the piano roll array
    piano_roll = np.zeros((64, piano_roll_length))
    
    # Fill in the piano roll array
    for note in transcription.instruments[0].notes:
        # Convert start and end times to frame indices
        start_frame = int(np.floor(note.start * fs))
        end_frame = int(np.ceil(note.end * fs))
        
        # Set the corresponding frames to 1 (or note.velocity for a velocity-sensitive representation)
        piano_roll[note.pitch, start_frame:end_frame] = 1  # Or use note.velocity
        
    roll = np.vstack([piano_roll[35:36, :], piano_roll[38:39, :], piano_roll[42:43, :], piano_roll[47:48, :], piano_roll[49:50, :]])
    return roll

# model

In [9]:
class DrumDemucs(pl.LightningModule):
    def __init__(self):
        super(DrumDemucs, self).__init__()

        self.loss_fn = auraloss.freq.MultiResolutionSTFTLoss(
                    fft_sizes=[1024, 2048, 4096],
                    hop_sizes=[256, 512, 1024],
                    win_lengths=[1024, 2048, 4096],
                    scale="mel", 
                    n_bins=150,
                    sample_rate=44100,
                    device="cuda"
                )

        self.loss_fn_2 = auraloss.time.SISDRLoss()

        self.loss_fn_3 = torch.nn.L1Loss()

        self.loss_used = 0

        sources = ['drum',
                   'noise',
                   ]
        
        self.demucs_mixer =  torchaudio.models.HDemucs(
            sources=sources,
            audio_channels=7,
            depth=6,
        )

        self.out_conv = nn.Conv1d(in_channels=7, out_channels=2, kernel_size=1)
        self.out = nn.Conv1d(in_channels=2, out_channels=2, kernel_size=1)      


    def compute_loss(self, outputs, ref_signals):
        loss = self.loss_fn(outputs, ref_signals) + self.loss_fn_2(outputs, ref_signals) +  self.loss_fn_3(outputs, ref_signals)
        return loss

    def forward(self, audio, drumroll):
        to_mix = torch.cat([audio, drumroll], axis=1)
        out = self.demucs_mixer(to_mix)
        out_2 = self.out_conv(out[:, 0, :, :])
        out_2 = self.out(out_2)
        # out_2 = torch.tanh(out_2)

        return out_2
    
    def training_step(self, batch, batch_idx):
        # training_step defines the train loop. It is independent of forward
        audio, drum, drumroll = batch
        
        outputs = self.forward(audio, drumroll)
        # print(outputs.size())

        if batch_idx % 64 == 0:
            input_signal = audio[0].cpu().detach().numpy().T
            generated_signal = outputs[0].cpu().detach().numpy().T
            drum_signal = drum[0].cpu().detach().numpy().T 
            wandb.log({'audio_input': [wandb.Audio(input_signal, caption="Input", sample_rate=44100)]})
            wandb.log({'audio_reference': [wandb.Audio(drum_signal, caption="Reference", sample_rate=44100)]})
            wandb.log({'audio_output': [wandb.Audio(generated_signal, caption="Output", sample_rate=44100)]})
             
            for i in range(5):
                wandb.log({f'drum_{i + 1}': [wandb.Audio(drumroll[0].cpu().detach().numpy()[i, :], caption="Output", sample_rate=44100)]})


        loss = self.compute_loss(outputs, drum)         

        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        
        return loss
    

    def configure_optimizers(self):
        # Define your optimizer and optionally learning rate scheduler here
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.99)
        return [optimizer], [scheduler]
        

In [10]:
def load_audio(path, start_point, filename):
    audio_tensors = []
    waveform, _ = torchaudio.load(f"{path}/{filename}")
    return waveform

def load_roll(path, start_point, frames):
    midi = path + '/mixture.wav.mid'
    transcription = pretty_midi.PrettyMIDI(midi)
    roll = turn_transcription_into_roll(transcription, frames)

    return torch.from_numpy(roll).float()


# SISNR

In [11]:
def separate_sources(
    model,
    mix,
    drumroll,
    segment=4.0,
    overlap=0.1,
    device=None,
):
    """
    Apply model to a given mixture. Use fade, and add segments together in order to add model segment by segment.

    Args:
        segment (int): segment length in seconds
        device (torch.device, str, or None): if provided, device on which to
            execute the computation, otherwise `mix.device` is assumed.
            When `device` is different from `mix.device`, only local computations will
            be on `device`, while the entire tracks will be stored on `mix.device`.
    """
    if device is None:
        device = mix.device
    else:
        device = torch.device(device)

    batch, channels, length = mix.shape

    chunk_len = int(sample_rate * segment * (1 + overlap))
    start = 0
    end = chunk_len
    overlap_frames = overlap * sample_rate
    fade = Fade(fade_in_len=0, fade_out_len=int(overlap_frames), fade_shape="linear")

    final = torch.zeros(batch, channels, length, device=device)

    while start < length - overlap_frames:
        chunk = mix[:, :, start:end]
        roll = drumroll[:, :, start:end]
        with torch.no_grad():
            out = model.forward(chunk, roll)
        out = fade(out)
        final[:, :, start:end] += out
        if start == 0:
            fade.fade_in_len = int(overlap_frames)
            start += int(chunk_len - overlap_frames)
        else:
            start += chunk_len
        end += chunk_len
        if end >= length:
            fade.fade_out_len = 0
    return final

In [12]:
class AudioData:
    def __init__(self, audio):
        self.audio = audio

In [13]:
for name in ['epoch_280', 'epoch_230','epoch_200','epoch_100']:
    try:
        try:
            os.mkdir(f"D:/Github/phd-drum-sep/analysis/demucs_small_model_analysis/results_{name}/")
        except:
            pass
    
        model = DrumDemucs.load_from_checkpoint(f'D:/Github/phd-drum-sep/analysis/demucs_small_model_analysis/checkpoint/{name}.ckpt')
        # model.to('cpu')
        model = model.eval()
        
        mus = musdb.DB(root="D:/Github/phd-drum-sep/Data/musdb18_stems/", subsets="test") 
        
        results = museval.EvalStore(frames_agg='median', tracks_agg='median')
        for track in tqdm(mus):
            path = "D:/Github/phd-drum-sep/Data/musdb18hq/"
            audio_path = path + 'test/' + track.name + "/"
        
        
            mixture_tensor = load_audio(audio_path, start_point,'mixture.wav').unsqueeze(0).to(model.device)
            drum_tensor = load_audio(audio_path, start_point,'drums.wav').unsqueeze(0).to(model.device)
            shape = mixture_tensor.shape[2]
            roll_tensor = load_roll(audio_path, start_point, shape).unsqueeze(0).to(model.device)
        
            seperated = separate_sources(model, mixture_tensor, roll_tensor, device='cuda')
        
            output_path = f"D:/Github/phd-drum-sep/analysis/demucs_small_model_analysis/results_{name}/"
            
            audio = seperated.squeeze(0).cpu().numpy()
            audio = np.swapaxes(audio,0,1)
            estimates = {'drums': audio, 'bass': audio}
        
            d = drum_tensor.squeeze(0).cpu().numpy()
            d = np.swapaxes(d,0,1)
            
            track.targets['drums'] = AudioData(d)
            track.targets['bass'] = AudioData(d)
            
            scores = museval.eval_mus_track(
                track, estimates, output_dir=f"{output_path}"
            )
        
            print(scores)
            results.add_track(scores)
            # break
        
        results.df.to_csv(f"{output_path}results.csv")
    
        del model
        del mus
        gc.collect()
    except Exception as e:
        print('error with model, skipping', name, e)

  2%|█▋                                                                                 | 1/50 [00:36<29:51, 36.56s/it]

drums           ==> SDR:   3.411  SIR: 229.041  ISR:   6.798  SAR:   3.322  
bass            ==> SDR:   3.411  SIR: 229.041  ISR:   6.798  SAR:   3.322  



  4%|███▎                                                                               | 2/50 [01:10<27:56, 34.93s/it]

drums           ==> SDR:   2.860  SIR: 181.808  ISR:   6.362  SAR:   0.352  
bass            ==> SDR:   2.860  SIR: 181.808  ISR:   6.362  SAR:   0.352  



  6%|████▉                                                                              | 3/50 [01:30<22:08, 28.26s/it]

...

drums           ==> SDR:  -1.712  SIR: 199.892  ISR:  -1.045  SAR:   1.473  
bass            ==> SDR:  -1.712  SIR: 199.892  ISR:  -1.045  SAR:   1.473  



 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [28:20<00:32, 32.71s/it]

drums           ==> SDR:  -2.862  SIR: 217.927  ISR:  -2.528  SAR:   9.074  
bass            ==> SDR:  -2.862  SIR: 217.927  ISR:  -2.528  SAR:   9.074  



100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [28:54<00:00, 34.70s/it]


drums           ==> SDR:  -2.754  SIR: 205.452  ISR:  -2.309  SAR:   4.909  
bass            ==> SDR:  -2.754  SIR: 205.452  ISR:  -2.309  SAR:   4.909  

